# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In this part, we import all the libraries we will use in this project and set some constants to represent the dataset.

In [1]:
# all import statements needed for the project
from math import radians,sin,cos,asin,sqrt
import bs4 
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import typing
import pytest
import ipytest
import re
import csv
from pandas.testing import assert_frame_equal
import os.path
from __future__ import annotations

In [2]:
# Set constants pointing to dataset
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
UBER_CSV = "uber_rides_sample.csv"
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculate distance and Add it to the dataset
Step 1: Calculate the distance between two coordinates using the haversine formula:  
Reference from Wikipedia  
![image](https://miro.medium.com/max/1392/0*mie5h2yduk6NmIym)  
Step 2: Add the distance  to the dataset as the new column

In [7]:
def calculate_distance(from_coord:tuple[float,float], to_coord:tuple[float,float]) -> float:
    #Convert decimal number to radian number
    from_lon,from_lat,to_lon,to_lat=map(radians,[from_coord[0],from_coord[1],to_coord[0],to_coord[1]])
    
    #using haversin formula
    dis_lon=to_lon-from_lon
    dis_lat=to_lat-from_lat
    haversine=sin(dis_lat/2)**2+cos(from_lat)*cos(to_lat)*sin(dis_lon/2)**2
    #6371 is the earth's radius(km)
    distance=6371*2*asin(sqrt(haversine))
    #round the result to two decimals
    return round(distance,2)

In [8]:
#TEST FOR calculate_distance()
#TEST BEGINS
@pytest.mark.parametrize(
    "from_coord,to_coord,expected",
    (
        ((-73.99981689,40.73835373),(-73.99951172,40.72321701),1.68),
        ((-73.994355,40.728225),(-73.99471,40.750325),2.46),
        ((-73.78808,40.642187),(-73.865042,40.725997),11.36),
        ((-73.992122,40.748577),(-73.806072,40.665272),18.21),
        ((-73.982321,40.768729),(-73.921654,40.660043),13.12),
    )
)

def test_calculate_distance(from_coord:tuple[float,float],to_coord,expected:tuple[float,float]) -> None:
    assert expected==calculate_distance(from_coord,to_coord)

ipytest.run("-vv", "-k", "test_calculate_distance")
#TEST ENDS

======================================= test session starts =======================================
platform win32 -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- C:\Program Files\Python310\python.exe
cachedir: .pytest_cache
rootdir: e:\projects\Python Final Project
collecting ... collected 5 items

tmpf9aobg6d.py::test_calculate_distance[from_coord0-to_coord0-1.68] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41772\1071625851.py PASSED [ 20%]
tmpf9aobg6d.py::test_calculate_distance[from_coord1-to_coord1-2.46] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41772\1071625851.py PASSED [ 40%]
tmpf9aobg6d.py::test_calculate_distance[from_coord2-to_coord2-11.36] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41772\1071625851.py PASSED [ 60%]
tmpf9aobg6d.py::test_calculate_distance[from_coord3-to_coord3-18.21] <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41772\1071625851.py PASSED [ 80%]
tmpf9aobg6d.py::test_calculate_distance[from_coord4-to_coord4-13.12] <- C:\Users\Lenovo\AppData\L

<ExitCode.OK: 0>

In [9]:
#Add distance to the dataset
def add_distance_column(df:pd.DataFrame) -> None:
    distance=[]
    for i,row in df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].iterrows():
        distance.append(calculate_distance((row['pickup_longitude'],row['pickup_latitude']),(row['dropoff_longitude'],row['dropoff_latitude'])))
    df['trip_distance']=distance
    #add distance to the new column 'trip_distance'

In [15]:
# TEST FOR add_distance_column
# TEST BEGINS
test_data = pd.DataFrame(
    {
        "pickup_longitude": [-73.99981689, -73.994355],
        "pickup_latitude": [40.73835373, 40.728225],
        "dropoff_longitude": [-73.99951172, -73.99471],
        "dropoff_latitude": [40.72321701, 40.750325],
    }
)
expected_data = pd.DataFrame(
    {
        "pickup_longitude": [-73.99981689, -73.994355],
        "pickup_latitude": [40.73835373, 40.728225],
        "dropoff_longitude": [-73.99951172, -73.99471],
        "dropoff_latitude": [40.72321701, 40.750325],
        "trip_distance": [1.68, 2.46],
    }
)


def test_add_distance():
    add_distance_column(test_data)
    assert_frame_equal(test_data, expected_data)


ipytest.run("-vv", "-k", "test_add_distance")
# TEST ENDS


======================================= test session starts =======================================
platform win32 -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0 -- C:\Program Files\Python310\python.exe
cachedir: .pytest_cache
rootdir: e:\projects\Python Final Project
collecting ... collected 6 items / 5 deselected / 1 selected

tmpd7w_pzty.py::test_add_distance <- C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41772\1575671478.py PASSED [100%]

================================= 1 passed, 5 deselected in 0.01s =================================


<ExitCode.OK: 0>

### Processing Uber Data

* Clean the data, including:
  * Remove unnecessary columns
  * Remove invalid data points
  * Remove trips that start and/or end outside the designated NEW_YORK_BOX_COORDS  
  

* Calculate the sample size for each month and so we can use it later to sample the Yellow taxi dataset

In [19]:
# load and clean data
def load_and_clean_uber_data(csv_file:str) -> pd.DataFrame:
    df = pd.read_csv(csv_file)
    # only pick the columns that will be used later
    df=df[['fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
    # Remove trips that start and/or end outside the designated coordinate box
    df = df[
        (df["pickup_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
        & (NEW_YORK_BOX_COORDS[0][1] <= df["pickup_longitude"])
        & (df["pickup_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
        & (NEW_YORK_BOX_COORDS[0][0] <= df["pickup_latitude"])
        & (df["dropoff_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
        & (NEW_YORK_BOX_COORDS[0][1] <= df["dropoff_longitude"])
        & (df["dropoff_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
        & (NEW_YORK_BOX_COORDS[0][0] <= df["dropoff_latitude"])
    ]
    # delete invalid data points where the fare_amount is 0
    return df[(df["fare_amount"] != 0)]

# get the whole cleaned dataset
def get_uber_data() -> pd.DataFrame:
    # get the cleaned data and add trip_distance column to it
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe


In [20]:
# calculate the sample size for each month
def get_sample_size(df:pd.DataFrame) -> dict:
    #group the data by the month
    df_month=df.groupby(df['pickup_datetime'].str[:7]).count()
    sample_size={}
    #store the month and sample size to the dict
    for month, size in df_month[['fare_amount']].iterrows():
        sample_size[month] = size.values[0]
    return sample_size

In [21]:
# run all functions and get the dataset
uber_data = get_uber_data()
sample_size=get_sample_size(uber_data)

In [23]:
# show the uber dataset
uber_data

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1.68
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,2.46
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,5.04
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,1.66
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,4.48
...,...,...,...,...,...,...,...
199995,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,0.11
199996,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1.88
199997,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,12.85
199998,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,3.54


### Processing Taxi Data


* Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
* Clean the data, including:
    * Remove unnecessary columns
    * Remove invalid data points (take a moment to consider what's invalid)
    * Normalize column names
    * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
* Sample the data so that we have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* Cache the sampling by saving it as a CSV file before moving onto the next file. 

In [24]:
url='https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2011-01.csv'
df = pd.read_csv(url)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41772\1139572720.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [25]:
df

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2011-01-29 02:38:35,2011-01-29 02:47:07,1,1.2,-74.005254,40.729084,1,N,-73.988697,40.727127,CSH,6.1,0.5,0.5,0.00,0.0,7.10
1,CMT,2011-01-28 10:38:19,2011-01-28 10:42:18,1,0.4,-73.968585,40.759171,1,N,-73.964336,40.764665,CSH,4.1,0.0,0.5,0.00,0.0,4.60
2,CMT,2011-01-28 23:49:58,2011-01-28 23:57:44,3,1.2,-73.980710,40.742390,1,N,-73.987028,40.729532,CSH,6.1,0.5,0.5,0.00,0.0,7.10
3,CMT,2011-01-28 23:52:09,2011-01-28 23:59:21,3,0.8,-73.993773,40.747329,1,N,-73.991378,40.750050,CSH,5.3,0.5,0.5,0.00,0.0,6.30
4,CMT,2011-01-28 10:34:39,2011-01-28 11:25:50,1,5.3,-73.991475,40.749936,1,N,-73.950237,40.775626,CSH,25.3,0.0,0.5,0.00,0.0,25.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13464991,CMT,2011-01-26 14:56:04,2011-01-26 15:02:52,1,1.2,-73.977775,40.787092,1,N,-73.982338,40.770560,CRD,5.7,0.0,0.5,1.25,0.0,7.45
13464992,CMT,2011-01-26 14:54:16,2011-01-26 15:00:48,1,1.0,-73.990756,40.743947,1,N,-74.005525,40.751080,CRD,5.3,0.0,0.5,0.87,0.0,6.67
13464993,CMT,2011-01-26 15:28:50,2011-01-26 15:52:50,1,3.2,-73.991282,40.755183,1,N,-74.002063,40.719173,CRD,13.7,0.0,0.5,2.13,0.0,16.33
13464994,CMT,2011-01-27 09:54:05,2011-01-27 10:05:16,1,1.2,-73.978428,40.750780,1,N,-73.973115,40.760543,CRD,7.3,0.0,0.5,1.17,0.0,8.97


In [ ]:
# find all the taxi csv file urls
def find_taxi_csv_urls() -> list:
    taxi_page = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(taxi_page.text, "html.parser")
    # match all the urls that are taxi csv files
    pattern = re.compile(
        "https:\/\/s3\.amazonaws\.com\/nyc-tlc\/trip\+data\/yellow_tripdata_\d{4}-\d{2}\.csv"
    )
    all_href = [a["href"] for a in soup.find_all("a")]
    all_url = pattern.findall("".join(all_href))
    all_url = all_url[54:60] + all_url[66:]
    return all_url

# Normalize all csv files to the same column names
def normalize_col_name(df:pd.DataFrame) -> pd.DataFrame:
    col_names=df.columns
    col_names=",".join(col_names).split(',')
    # Set all the names to the following new names
    col_newname=['pickup_datetime','dropoff_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','tip_amount','total_amount']
    col_oldname=[]
    # find all the old col names 
    pattern1=".*time.*"
    pattern2=".*lon.*"
    pattern3=".*lat.*"
    pattern4=".*[Tt]ip.*"
    pattern5=".*[Tt]otal.*"
    for i in col_names:
        if(re.findall(pattern1,i) != []):
            col_oldname.append(re.findall(pattern1,i)[0])
        if(re.findall(pattern2,i) != [] ):
            col_oldname.append(re.findall(pattern2,i)[0])
        if(re.findall(pattern3,i) != [] ):
            col_oldname.append(re.findall(pattern3,i)[0])
        if(re.findall(pattern4,i) != [] ):
            col_oldname.append(re.findall(pattern4,i)[0])
        if(re.findall(pattern5,i) != [] ):
            col_oldname.append(re.findall(pattern5,i)[0])

    name_dict=dict(zip(col_oldname,col_newname))
    # change col names
    df.rename(columns=name_dict,inplace=True)
    # We only select part of the columns and remove unnecessary columns
    return df[col_newname]
    
# load and clean data
# if the csv files has been saved before, then load it directly from the computer
def get_and_clean_month_taxi_data(url: str) -> pd.DataFrame:
    month = url[59:66]
    sample = sample_size[month]
    file_exists = os.path.exists(f"taxi_{month}.csv")
    # if file already saved, then load it from the computer
    if file_exists == True:
        df = pd.read_csv(f"taxi_{month}.csv")
    # if file haven't been saved, then load it from the URL.
    else:
        df = pd.read_csv(url)
        df = df[(df["total_amount"] != 0)]
        df = df[
            (df["pickup_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
            & (NEW_YORK_BOX_COORDS[0][1] <= df["pickup_longitude"])
            & (df["pickup_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
            & (NEW_YORK_BOX_COORDS[0][0] <= df["pickup_latitude"])
            & (df["dropoff_longitude"] <= NEW_YORK_BOX_COORDS[1][1])
            & (NEW_YORK_BOX_COORDS[0][1] <= df["dropoff_longitude"])
            & (df["dropoff_latitude"] <= NEW_YORK_BOX_COORDS[1][0])
            & (NEW_YORK_BOX_COORDS[0][0] <= df["dropoff_latitude"])
        ]
        # Normalize the col names and delete other unnecessary columns
        df=normalize_col_name(df)
        # Sample the data so that we have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
        df = df.sample(n=sample, random_state=1)
        df.to_csv(f"taxi_{month}.csv")
    return df


def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        all_taxi_dataframes.append(dataframe)

    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data


In [ ]:
taxi_data = get_and_clean_taxi_data()

### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    

def clean_month_weather_data_daily(csv_file):
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []

    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]

    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)

    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data


In [ ]:

hourly_weather, daily_weather = load_and_clean_weather_data()


## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)


In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO 
"""


In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)


In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass


### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def dataframes_to_table():
    raise NotImplemented()


In [ ]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

dataframes_to_table(table_name_to_dataframe)


## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()


### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""


In [ ]:
engine.execute(QUERY_N).fetchall()


In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")


## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))

    values = "..."  # use the dataframe to pull out values needed to plot

    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")

    plt.show()


In [ ]:
plot_visual_n(some_dataframe)
